In [135]:
# importing module
from pandas import *
from statistics import mean
import numpy as np
from helper760 import read_inputs
import tensorflow as tf

test_case = [0,1,2,3]

In [136]:
Clininc_Data,Outcome_Data,CT_Data = read_inputs()

# Test Case 0 - knn CT-To-Cancer Prediction
This is a test run perform prediction of cancer using CT Data. 

In [137]:
if 0 in test_case:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import KFold
    n_split = 5

    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)
    X_normed = (X - X.min(0)) / X.ptp(0)
    y = np.array(Outcome_Data[21])
    y = y.astype(int)

    k_range = range(1,11,1)
    detection_rate = []
    wrong_diagnosis = []
    accuracy_rate = []
    
    for k in k_range:
        temp_sum = 0
        for train_index,test_index in KFold(n_split).split(X):

            x_train,x_test = X_normed[train_index,:], X_normed[test_index,:]
            y_train,y_test = y[train_index], y[test_index]


            knn_clf = KNeighborsClassifier(n_neighbors=k)
            knn_clf.fit(x_train, y_train)

            y_predict = knn_clf.predict(x_test)

            count_right = 0
            count_wrong = 0

            detected_cancer = 0
            detected_wrong = 0
            total_cancer = 0
            for i in range((y_predict.shape)[0]):
                if y_test[i] == 1:
                    total_cancer = total_cancer + 1


                if y_predict[i] == y_test[i]:
                    if y_test[i] != 0:
                        detected_cancer = detected_cancer + 1
                    count_right = count_right + 1
                else:
                    if y_test[i] == 0:
                        detected_wrong = detected_wrong + 1
                    count_wrong = count_wrong + 1

            temp_sum = temp_sum + detected_cancer/total_cancer
        detection_rate.append(temp_sum/n_split)
        wrong_diagnosis.append(detected_wrong)
        accuracy_rate.append(count_right/(count_right+count_wrong))
    
    final_idx = detection_rate.index(max(detection_rate))
    print("The best n_neighbours is: " + str(final_idx+1)+". The cv=5 cancer detection rate is: "+str(detection_rate[final_idx])+". The wrong diagnosis number is: "+
    str(wrong_diagnosis[final_idx]) + ". The accuracy is: "+str(accuracy_rate[final_idx]))




The best n_neighbours is: 1. The cv=5 cancer detection rate is: 0.15039061485405567. The wrong diagnosis number is: 251. The accuracy is: 0.7885032537960954


# Test Case 1 - knn CT+Clinic-To-Cancer Prediction
This is a test run perform prediction of cancer using CT Data and Clinic Data. 

In [138]:
if 1 in test_case:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import KFold
    n_split = 5

    X1 = np.array(CT_Data)
    X1 = X1.T
    X1 = X1.astype(float)

    X2 = np.array(Clininc_Data)
    X2 = X2[4:14,:]
    X2 = X2.T
    X2 = X2.astype(float)

    X = np.column_stack((X1,X2))
    X_normed = X

    y = np.array(Outcome_Data[21])
    y = y.astype(int)

    k_range = range(1,11,1)
    detection_rate = []
    wrong_diagnosis = []
    accuracy_rate = []
    
    for k in k_range:
        temp_sum = 0
        for train_index,test_index in KFold(n_split).split(X):

            x_train,x_test = X_normed[train_index,:], X_normed[test_index,:]
            y_train,y_test = y[train_index], y[test_index]


            knn_clf = KNeighborsClassifier(n_neighbors=k, weights='distance')
            knn_clf.fit(x_train, y_train)

            y_predict = knn_clf.predict(x_test)

            count_right = 0
            count_wrong = 0

            detected_cancer = 0
            detected_wrong = 0
            total_cancer = 0
            for i in range((y_predict.shape)[0]):
                if y_test[i] == 1:
                    total_cancer = total_cancer + 1


                if y_predict[i] == y_test[i]:
                    if y_test[i] != 0:
                        detected_cancer = detected_cancer + 1
                    count_right = count_right + 1
                else:
                    if y_test[i] == 0:
                        detected_wrong = detected_wrong + 1
                    count_wrong = count_wrong + 1

            temp_sum = temp_sum + detected_cancer/total_cancer
        detection_rate.append(temp_sum/n_split)
        wrong_diagnosis.append(detected_wrong)
        accuracy_rate.append(count_right/(count_right+count_wrong))
    
    final_idx = detection_rate.index(max(detection_rate))
    print(detection_rate)
    print("The best n_neighbours is: " + str(final_idx+1)+". The cv=5 cancer detection rate is: "+str(detection_rate[final_idx])+". The wrong diagnosis number is: "+
    str(wrong_diagnosis[final_idx]) + ". The accuracy is: "+str(accuracy_rate[final_idx]))

[0.14474326692886658, 0.14474326692886658, 0.05729497681590724, 0.05123951263157571, 0.021599041159625618, 0.015377072661358673, 0.00448230650035408, 0.003347920926609239, 0.0014236133022023096, 0.0014236133022023096]
The best n_neighbours is: 1. The cv=5 cancer detection rate is: 0.14474326692886658. The wrong diagnosis number is: 224. The accuracy is: 0.7982646420824295


# Test Case 2 - knn CT-To-Disease Prediction
This is a test run perform prediction of diseases using CT Data 

In [139]:
if 2 in test_case:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import KFold
    n_split = 5

    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)
    #X_normed = (X - X.min(0)) / X.ptp(0)
    X_normed = X
    y = np.array(Outcome_Data)
    y = y.T
    y = y.astype(int)
    y = y[:,[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]]
    k_range = range(1,11,1)
    detection_rate = []
    wrong_diagnosis = []
    accuracy_rate = []
    right_arr = []
    total_arr = []
    
    for k in k_range:
        temp_sum = 0
        for train_index,test_index in KFold(n_split).split(X):

            x_train,x_test = X_normed[train_index,:], X_normed[test_index,:]
            y_train,y_test = y[train_index,:], y[test_index,:]


            knn_clf = KNeighborsClassifier(n_neighbors=k, weights='distance')
            knn_clf.fit(x_train, y_train)

            y_predict = knn_clf.predict(x_test)

            count_right = 0
            count_wrong = 0

            detected_disease = 0
            detected_wrong = 0
            total_disease = 0
            for i in range((y_predict.shape)[0]):
                for j in range((y_predict.shape)[1]):
                    if y_test[i,j] == 1:
                        total_disease = total_disease + 1


                    if y_predict[i,j] == y_test[i,j]:
                        if y_test[i,j] != 0:
                            detected_disease = detected_disease + 1
                        count_right = count_right + 1
                    else:
                        if y_test[i,j] == 0:
                            detected_wrong = detected_wrong + 1
                        count_wrong = count_wrong + 1

            temp_sum = temp_sum + detected_disease/total_disease
        detection_rate.append(temp_sum/n_split)
        wrong_diagnosis.append(detected_wrong)
        accuracy_rate.append(count_right/(count_right+count_wrong))
        right_arr.append(detected_disease)
        total_arr.append(total_disease)
    
    final_idx = detection_rate.index(max(detection_rate))
    print(detection_rate)
    print("The best n_neighbours is: " + str(final_idx+1)+". The cv=5 disease detection rate is: "+str(detection_rate[final_idx])+". The wrong diagnosis number is: "+
    str(wrong_diagnosis[final_idx]) + ". Correct disease diagnosis is:" + str(right_arr[final_idx])+ " Total disease in sample: "+str(total_arr[final_idx])+". The accuracy is: "+str(accuracy_rate[final_idx]))


[0.2034024542682455, 0.2034024542682455, 0.14624740846631176, 0.14342857894580735, 0.11395130498141685, 0.11348449009416536, 0.1003033940368709, 0.09742991234058312, 0.09095690531676255, 0.08703721398408636]
The best n_neighbours is: 1. The cv=5 disease detection rate is: 0.2034024542682455. The wrong diagnosis number is: 1289. Correct disease diagnosis is:298 Total disease in sample: 1194. The accuracy is: 0.8922796292644449


# Test Case 3 - knn CT+Clinic-To-Disease Prediction
This is a test run perform prediction of diseases using CT Data and Clinic Data

In [140]:
if 3 in test_case:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import KFold
    n_split = 5

    X1 = np.array(CT_Data)
    X1 = X1.T
    X1 = X1.astype(float)

    X2 = np.array(Clininc_Data)
    X2 = X2[4:14,:]
    X2 = X2.T
    X2 = X2.astype(float)

    X = np.column_stack((X1,X2))
    X_normed = X
    y = np.array(Outcome_Data)
    y = y.T
    y = y.astype(int)
    y = y[:,[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]]
    k_range = range(1,11,1)
    detection_rate = []
    wrong_diagnosis = []
    accuracy_rate = []
    right_arr = []
    total_arr = []
    
    for k in k_range:
        temp_sum = 0
        for train_index,test_index in KFold(n_split).split(X):

            x_train,x_test = X_normed[train_index,:], X_normed[test_index,:]
            y_train,y_test = y[train_index,:], y[test_index,:]


            knn_clf = KNeighborsClassifier(n_neighbors=k, weights='distance')
            knn_clf.fit(x_train, y_train)

            y_predict = knn_clf.predict(x_test)

            count_right = 0
            count_wrong = 0

            detected_disease = 0
            detected_wrong = 0
            total_disease = 0
            for i in range((y_predict.shape)[0]):
                for j in range((y_predict.shape)[1]):
                    if y_test[i,j] == 1:
                        total_disease = total_disease + 1


                    if y_predict[i,j] == y_test[i,j]:
                        if y_test[i,j] != 0:
                            detected_disease = detected_disease + 1
                        count_right = count_right + 1
                    else:
                        if y_test[i,j] == 0:
                            detected_wrong = detected_wrong + 1
                        count_wrong = count_wrong + 1

            temp_sum = temp_sum + detected_disease/total_disease
        detection_rate.append(temp_sum/n_split)
        wrong_diagnosis.append(detected_wrong)
        accuracy_rate.append(count_right/(count_right+count_wrong))
        right_arr.append(detected_disease)
        total_arr.append(total_disease)
    
    final_idx = detection_rate.index(max(detection_rate))
    print(detection_rate)
    print("The best n_neighbours is: " + str(final_idx+1)+". The cv=5 disease detection rate is: "+str(detection_rate[final_idx])+". The wrong diagnosis number is: "+
    str(wrong_diagnosis[final_idx]) + ". Correct disease diagnosis is:" + str(right_arr[final_idx])+ " Total disease in sample: "+str(total_arr[final_idx])+". The accuracy is: "+str(accuracy_rate[final_idx]))

[0.20211082994677704, 0.20211082994677704, 0.1461819744824923, 0.14422451055652277, 0.1145132904045221, 0.11335224108637063, 0.10094027000068824, 0.09811086504888675, 0.09114890980652364, 0.08726213297907029]
The best n_neighbours is: 1. The cv=5 disease detection rate is: 0.20211082994677704. The wrong diagnosis number is: 1275. Correct disease diagnosis is:294 Total disease in sample: 1194. The accuracy is: 0.8927726286728456
